In [3]:
#import libraries
from time import clock
from osgeo import gdal
import os , sys


In [5]:
# hdf to tif, project=Albers
start = clock()
files_names = [f for f in os.listdir(r_data_dir) if f.endswith('.hdf')]
for i in files_names:
    name=  "\"HDF4_EOS:EOS_GRID:"+ data_dir + i+ ":MOD_Grid_Monthly_500m_DB_BA:Burn Date\""
    #print name
    #b=name[39:46]+ "_"+ name[47:53]
    b=name[45:53]+ "_"+ name[54:60]
    year= name[46:50]
    #print b, year
    if not os.path.exists(r_data_dir+"/"+year):
      os.makedirs(r_data_dir+"/"+year)
    dset = data_dir +year+"/"+b +".tif"
    #print dset
    #os.system("gdal_translate -of GTiff -a_srs \"+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs\" " + name + " " + dset)
    os.system("gdal_translate -of GTiff " + name + " " + dset)
    #outt="+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs"
    #os.system("gdalwarp -s_srs \" PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]")
    #out_proj = 'PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0],AUTHORITY["ESRI","102039"]]'
    #input_proj="PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]"
    #os.system("gdalwarp -of GTiff -s_srs \" PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]\"  -t_srs"  + out_proj +" "+ name + " " + dset)
    #os.system("gdal_translate -of GTiff -a_srs " + out_proj +" "+ name + " " + dset)
    #os.system("gdalwarp -of GTiff  -s_srs \" PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]"\ -a_srs " + out_proj +" "+ name + " " + dset)
    #os.system("gdal_translate -of GTiff -a_srs \"PROJCS['USA_Contiguous_Albers_Equal_Area_Conic_USGS_version',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-96.0],PARAMETER['Standard_Parallel_1',29.5],PARAMETER['Standard_Parallel_2',45.5],PARAMETER['Latitude_Of_Origin',23.0],UNIT['Meter',1.0]]", resampling_type="NEAREST\" " + name + " " + dset)
    #os.system("gdal_translate -of GTiff -a_srs \"+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs\" " + name + " " + dset)
# Replace a layer/table view name with a path to a dataset (which can be a layer file) or create the layer/table view within the script
# The following inputs are layers or table views: "A2010.tif"
#arcpy.ProjectRaster_management(in_raster="A2010.tif", out_raster="C:/Users/sepid/Documents/ArcGIS/Default1.gdb/A2010_ProjectRaster", out_coor_system="PROJCS['USA_Contiguous_Albers_Equal_Area_Conic_USGS_version',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-96.0],PARAMETER['Standard_Parallel_1',29.5],PARAMETER['Standard_Parallel_2',45.5],PARAMETER['Latitude_Of_Origin',23.0],UNIT['Meter',1.0]]", resampling_type="NEAREST", cell_size="463.312716527917 463.312716527917", geographic_transform="", Registration_Point="", in_coor_system="PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]")
print round (time.clock()-start,2),'second' 

1.26 second


In [11]:
out_proj = str('PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0],AUTHORITY["ESRI","102039"]]')


In [12]:
in_proj= str("PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]")


In [36]:

# trying Gdal_warp


start = clock()
data_dir = "test/2009"
r_data_dir = "./" + data_dir
files_names = [f for f in os.listdir(data_dir) if f.endswith('.tif')]
for i in files_names:
    print i
    #os.system("gdalwarp %s %s -of GTiff -t_srs %s %s -overwrite  " %(i,i,outt,dstfile))
    #os.system("gdalwarp -overwrite %s %s -of GTiff -s_srs %s -t_srs %s   " %(i,i+"new",input_proj,outt))
    #os.system("gdaltransform -s_srs %s -t_srs %s " %(input_proj,outt))
    #os.system("gdaltransform -s_srs %s -t_srs %s %s" %(input_proj,outt,dstfile2 ))
    #os.system("gdalwarp  -t_srs \"+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs\"  -s_srs \" PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]\"" )               

A2016122_h08v05.tif


In [42]:
#from osgeo import gdal

'''filename = r"C:\Users\sepid\Documents\fire_event_py\test\A2016122_h08v05.tif"
input_raster = gdal.Open(filename)
output_raster = r"C:\Users\sepid\Documents\fire_event_py\test\test2\A2016122_h08v05.tif"
os.system( "gdalwarp(output_raster,input_raster,dstSRS='+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs'")
'''

1

In [41]:
'''
os.system("gdalbuildvrt -of GTiff -srcnodata -1 2010001_h08v05.tif myVrt.vrt")
os.system("gdal_translate -of GTiff -scale -a_nodata -1 myVrt.vrt /tifs/test3/2010/output.tif")
'''

1

In [29]:
'''

import os
data_dir = "./tifs/test1/2009/"
# Define input and output files
inputfiles =  "2009305_h08v05.tif 2009335_h08v05.tif"
outputfile =  "./tifs/test1/2009/mergedfile.tif"

# Generate the command
command = "gdal_merge.py %s -o %s" % (inputfiles, outputfile)
os.system(command)
'''

1